In [3]:
pip install nltk scikit-learn gensim

     ---------------------------------------- 24.0/24.0 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 KB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\yemia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [10]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
# Download the punkt tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yemia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Load the data
data = pd.read_csv('sent_train.csv')
data.head()


,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0


In [6]:
# Install the stopwords resource
nltk.download('stopwords')

# Data cleaning
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\$[A-Za-z]+', '', text)  # Remove stock symbols
    text = re.sub(r'\W+', ' ', text.lower())  # Remove non-word characters and convert to lowercase
    return text.strip()

data['cleaned_text'] = data['text'].apply(clean_text)

# Remove stop words and perform stemming
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

data['processed_text'] = data['cleaned_text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split() if word not in stop_words]))

# Now, 'processed_text' column contains the preprocessed text data
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yemia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label,cleaned_text,processed_text
0,$BYND - JPMorgan reels in expectations on Beyo...,0,jpmorgan reels in expectations on beyond meat,jpmorgan reel expect beyond meat
1,$CCL $RCL - Nomura points to bookings weakness...,0,nomura points to bookings weakness at carnival...,nomura point book weak carniv royal caribbean
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0,cemex cut at credit suisse j p morgan on weak ...,cemex cut credit suiss j p morgan weak build o...
3,$ESS: BTIG Research cuts to Neutral https://t....,0,btig research cuts to neutral,btig research cut neutral
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0,funko slides after piper jaffray pt cut,funko slide piper jaffray pt cut


In [8]:
# Tokenize the preprocessed text (sentences)
data['tokenized_text'] = data['processed_text'].apply(word_tokenize)

data.head()

,text,label,cleaned_text,processed_text,tokenized_text
0,$BYND - JPMorgan reels in expectations on Beyo...,0,jpmorgan reels in expectations on beyond meat,jpmorgan reel expect beyond meat,"[jpmorgan, reel, expect, beyond, meat]"
1,$CCL $RCL - Nomura points to bookings weakness...,0,nomura points to bookings weakness at carnival...,nomura point book weak carniv royal caribbean,"[nomura, point, book, weak, carniv, royal, car..."
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0,cemex cut at credit suisse j p morgan on weak ...,cemex cut credit suiss j p morgan weak build o...,"[cemex, cut, credit, suiss, j, p, morgan, weak..."
3,$ESS: BTIG Research cuts to Neutral https://t....,0,btig research cuts to neutral,btig research cut neutral,"[btig, research, cut, neutral]"
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0,funko slides after piper jaffray pt cut,funko slide piper jaffray pt cut,"[funko, slide, piper, jaffray, pt, cut]"


In [9]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=data['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)
# Function to calculate the average word embedding for a text sample
def get_average_embedding(words):
    embeddings = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        # If no word in the text sample is present in the word2vec model, return a zero vector
        return np.zeros(word2vec_model.vector_size)

# Get average word embeddings for each text sample
X = [get_average_embedding(words) for words in data['tokenized_text']]
# Prepare target variable
y = data['label']

In [11]:
# Prepare target variable
y = data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)

# Train the classifier
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the model
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       285
           1       0.50      0.03      0.06       391
           2       0.65      1.00      0.79      1233

    accuracy                           0.65      1909
   macro avg       0.38      0.34      0.28      1909
weighted avg       0.52      0.65      0.52      1909



C:\Users\yemia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yemia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yemia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: Unde

# Evaluating Validation Data

In [12]:
# Load the data
val_data = pd.read_csv('sent_valid.csv')
val_data.head()

,text,label
0,$ALLY - Ally Financial pulls outlook https://t...,0
1,"$DELL $HPE - Dell, HPE targets trimmed on comp...",0
2,$PRTY - Moody's turns negative on Party City h...,0
3,$SAN: Deutsche Bank cuts to Hold,0
4,$SITC: Compass Point cuts to Sell,0


In [14]:
val_data['cleaned_text'] = val_data['text'].apply(clean_text)
val_data['processed_text'] = val_data['cleaned_text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split() if word not in stop_words]))

# Tokenize the preprocessed text (sentences)
val_data['tokenized_text'] = val_data['processed_text'].apply(word_tokenize)

val_data.head()

,text,label,cleaned_text,processed_text,tokenized_text
0,$ALLY - Ally Financial pulls outlook https://t...,0,ally financial pulls outlook,alli financi pull outlook,"[alli, financi, pull, outlook]"
1,"$DELL $HPE - Dell, HPE targets trimmed on comp...",0,dell hpe targets trimmed on compute headwinds,dell hpe target trim comput headwind,"[dell, hpe, target, trim, comput, headwind]"
2,$PRTY - Moody's turns negative on Party City h...,0,moody s turns negative on party city,moodi turn neg parti citi,"[moodi, turn, neg, parti, citi]"
3,$SAN: Deutsche Bank cuts to Hold,0,deutsche bank cuts to hold,deutsch bank cut hold,"[deutsch, bank, cut, hold]"
4,$SITC: Compass Point cuts to Sell,0,compass point cuts to sell,compass point cut sell,"[compass, point, cut, sell]"


In [15]:

# Get average word embeddings for each text sample
val_X = [get_average_embedding(words) for words in val_data['tokenized_text']]
# Prepare target variable
val_y = val_data['label']


# Model prediction on val data
val_y_pred = svm_classifier.predict(val_X)
# Model evaluation
val_accuracy = accuracy_score(val_y, val_y_pred)
print(f"Accuracy: {val_accuracy:.2f}")
print(classification_report(val_y, val_y_pred))

Accuracy: 0.66
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       347
           1       0.44      0.04      0.07       475
           2       0.66      0.99      0.80      1566

    accuracy                           0.66      2388
   macro avg       0.37      0.34      0.29      2388
weighted avg       0.52      0.66      0.54      2388



C:\Users\yemia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yemia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yemia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: Unde